In [1]:
%load_ext autoreload
%autoreload 2

import os
os.chdir("../../")
print(os.getcwd())

C:\Users\Milosz\Desktop\python\thesis-recsys


In [2]:
import pandas as pd
import numpy as np
import functools
import operator
import json
import matplotlib.pyplot as plt
from tqdm import tqdm

import torch_geometric
import torch_geometric.transforms as T
from torch_geometric import nn
from torch_geometric.sampler import NegativeSampling
from torch_geometric.loader import LinkNeighborLoader, NeighborLoader
from torch_geometric.data import HeteroData
from torch_geometric.utils import to_scipy_sparse_matrix

import torch
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

from sklearn.metrics import roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler

from src.utils import *

torch.set_printoptions(precision=2, sci_mode=False)
torch.manual_seed(715037601397000) # used to train gnn_02.pth

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [4]:
app_meta = load_data_from_csv("data/graph_app_tags_stacked.csv")

In [5]:
app_features = np.zeros((1797, 425), dtype=np.float32)

for i, (_, x) in enumerate(app_meta['tags_id'].iteritems()):
    x = np.fromstring(x[1:-1], dtype=int, sep=',')
    app_features[i, x] = 1

C:\Users\Milosz\AppData\Local\Temp\ipykernel_18732\612704181.py:3: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for i, (_, x) in enumerate(app_meta['tags_id'].iteritems()):


In [6]:
#app_features = load_data_from_csv("data/graph_app_features.csv")

In [7]:
# real_cols = ['positive_ratio', 'user_reviews', 'price_final', 'price_original', 'discount']

# scaler = StandardScaler()
# app_features_norm = scaler.fit_transform(app_features[real_cols].numpy())

In [8]:
def load_graph(df: pd.DataFrame, n_users: int, n_items: int) -> HeteroData:
    """
    Loads a graph data structure from a pandas DataFrame.
    
    Parameters:
        - df (pd.DataFrame): The input DataFrame containing the graph data.

    Returns:
        - HeteroData: A heterogeneous graph data object representing the input graph.

    Example:
        >>> import pandas as pd
        >>> df = pd.DataFrame({'user_id': [1, 2, 3], 'app_id': [4, 5, 6], 'is_recommended': [1, 0 ,1]})
        >>> graph = load_graph(df)
    """
    
    data = HeteroData()
    
    data['user'].x = torch.ones(n_users, 1)
    data['user'].n_id = torch.arange(n_users)
    
    data['app'].x = torch.from_numpy(app_features)
    data['app'].n_id = torch.arange(n_items)
    
    edge_index = torch.tensor([df['user_id'].values, df['app_id'].values])
    edge_label = torch.tensor(df['is_recommended'].values, dtype=torch.long)

    data['user', 'recommends', 'app'].edge_index = edge_index
    data['user', 'recommends', 'app'].edge_label = edge_label
    
    return data

In [9]:
def transform_graph(data: HeteroData) -> HeteroData:
    """
    Applies a transformation to a heterogeneous graph data object.

    Parameters:
        data: The input graph data object to be transformed.

    Returns:
        HeteroData: A new heterogeneous graph data object resulting from the transformation.

    Example:
        >>> transformed_data = transform_graph(data)
    """
    transform = T.Compose([T.ToUndirected()])
    return transform(data)

In [10]:
def init_edge_loader(data: HeteroData, **kwargs) -> NeighborLoader:
    """
    Initializes a neighbor loader for edge-based data in a heterogeneous graph.
    Firstly we sample `batch_size` edges and then sample at most `num_neighbors[0]`
    neighboring edges at first hop and at most `num_neighbors[1]` at second hop. 
    Value returned by next(iter(loader)) is a subgraph of `data` graph containing
    only sampled edges and congruent nodes.

    Args:
        data (HeteroData): The input heterogeneous graph data object.
        **kwargs: Additional keyword arguments for configuring the loader.

    Returns:
        NeighborLoader: A neighbor loader for the specified edge-based data.

    Example:
        >>> loader = init_edge_loader(data, num_neighbors=5, neg_sampl=0.2, bs=32, shuffle=True)
    """
    
    eli = data['user', 'recommends', 'app'].edge_label_index
    el = data['user', 'recommends', 'app'].edge_label
    
    loader = LinkNeighborLoader(
        data=data,
        num_neighbors=kwargs['num_neighbors'],
        neg_sampling_ratio=kwargs['neg_sampl'],
        edge_label_index=(('user', 'recommends', 'app'), eli),
        edge_label=el,
        batch_size=kwargs['bs'],
        shuffle=kwargs['shuffle'],
    )
    return loader

In [11]:
def get_sparse_adj_matr(data):
    # Extract sparse adjacency matrix with message passing edges
    mp_edges = data['user', 'recommends', 'app']['edge_index']
    mp_matrix = to_scipy_sparse_matrix(mp_edges, num_nodes=n_users).tocsr()

    # Extract sparse adjacency matrix with validation edges
    true_mask = data['user', 'recommends', 'app']['edge_label'].nonzero().flatten()
    val_edges = data['user', 'recommends', 'app']['edge_label_index'][:, true_mask]
    val_matrix = to_scipy_sparse_matrix(val_edges, num_nodes=n_users).tocsr()
    
    return mp_matrix, val_matrix

In [12]:
train_df = load_data_from_csv("data/graph_train.csv")
test_df = load_data_from_csv("data/graph_test.csv")

In [13]:
df = pd.concat([train_df, test_df])
n_users, n_items = df.user_id.nunique(), df.app_id.nunique()
data = load_graph(df, n_users, n_items)
data = transform_graph(data)

C:\Users\Milosz\AppData\Local\Temp\ipykernel_18732\1547020800.py:25: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:233.)
  edge_index = torch.tensor([df['user_id'].values, df['app_id'].values])


In [14]:
random_split = T.RandomLinkSplit(
    num_val=0.3,
    num_test=0.0,
    add_negative_train_samples=False,
    neg_sampling_ratio=2.0,
    disjoint_train_ratio=0.3,
    edge_types=('user', 'recommends', 'app'),
    rev_edge_types=('app', 'rev_recommends', 'user')
)
train_data, val_data, _ = random_split(data)

In [15]:
train_loader = init_edge_loader(train_data, num_neighbors=[20, 10], neg_sampl=2.0, bs=1024, shuffle=True, drop_last=True)
val_loader = init_edge_loader(val_data, num_neighbors=[20, 10], neg_sampl=2.0, bs=256, shuffle=False, drop_last=True)

In [16]:
mp_matrix, val_matrix = get_sparse_adj_matr(val_data)

In [17]:
# Dataloader:
#  - user: x->attributes of sampled nodes, n_id->mapping of sampled nodes to ids from whole graph
#  - app: x->attributes of sampled nodes, n_id->mapping of sampled nodes to ids from whole graph
#  - (user recommends app): 
#      edge_index -> sampled edges with batch ids with neighbors
#      edge_label -> labels of edges which will be evaluated, size of batch size
#      e_id -> mapping of sampled edges to ids from whole graph, refers to ?????
#      input_id -> mapping of sampled edges to ids from whole graph, refers to edge_label_index
#      edge_label_index -> edge index, ids of nodes in sampled graph which will be evaluated


# To validate nodes first get sampled nodes ids from edge_label_index, then map them to whole graph
# using n_ids of user and app and then check if such edge exists in dataframe

In [18]:
def train_fn(train_data: HeteroData, test_data: HeteroData):
    pass

In [19]:
class GNN(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = nn.SAGEConv((hidden_channels, hidden_channels), hidden_channels, normalize=True)
        self.conv2 = nn.SAGEConv((hidden_channels, hidden_channels), out_channels, normalize=False)
        
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x
    
    
class Classifier(torch.nn.Module):
    def forward(self, x_user, x_app, edge_label_index):
        x_user = x_user[edge_label_index[0]]
        x_app = x_app[edge_label_index[1]]
        return (x_user * x_app).sum(dim=-1)


class Model(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels, metadata):
        super().__init__()
        
        self.user_emb = torch.nn.Embedding(n_users, hidden_channels)
        self.app_emb = torch.nn.Embedding(n_items, hidden_channels)
        self.app_lin = torch.nn.Linear(425, hidden_channels)
        
        self.gnn = GNN(hidden_channels=hidden_channels, out_channels=out_channels)
        self.gnn = nn.to_hetero(self.gnn, metadata=metadata, aggr='sum')
        
        self.clf = Classifier()
        
    def forward(self, batch):  
        x_dict = {
          "user": self.user_emb(batch['user'].n_id),
          "app": self.app_emb(batch['app'].n_id) + self.app_lin(batch['app'].x),
        } 
        
        x_dict = self.gnn(x_dict, batch.edge_index_dict)
        pred = self.clf(
            x_dict["user"],
            x_dict["app"],
            batch['user', 'recommends', 'app'].edge_label_index,
        )
        return pred
    
    def evaluate(self, batch):
        x_dict = {
          "user": self.user_emb(batch['user'].n_id),
          "app": self.app_emb(batch['app'].n_id) + self.app_lin(batch['app'].x),
        } 

        x_dict = self.gnn(x_dict, batch.edge_index_dict)

        return x_dict

def xavier_init(m):
    if isinstance(m, torch.nn.Linear) or isinstance(m, torch_geometric.nn.dense.linear.Linear):
        torch.nn.init.xavier_normal_(m.weight, gain=1.41)
        if m.bias is not None:
            torch.nn.init.zeros_(m.bias)
    
model = Model(hidden_channels=32, out_channels=32, metadata=train_data.metadata())
model.apply(xavier_init)
model = model.to(device)

In [20]:
criterion = torch.nn.BCEWithLogitsLoss()
#optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-2)
#optimizer = torch.optim.SGD(params=model.parameters(), lr=1e-1, momentum=0.9)
optimizer = torch.optim.RMSprop(params=model.parameters(), lr=0.001, momentum=0.9)
#writer = SummaryWriter()

In [21]:
def train(n_epochs, print_loss=500):
    model.train()
    
    for epoch in range(n_epochs):
        running_loss = 0.
        for i_batch, batch in enumerate(tqdm(train_loader)):
            batch = batch.to(device)
            
            y_pred = model(batch)
            y_true = batch['user', 'recommends', 'app'].edge_label
            loss = criterion(y_pred, y_true.float())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            if not ((i_batch+1) % print_loss):
                last_loss = running_loss / print_loss
                #writer.add_scalar("Loss/train", last_loss, epoch*len(train_loader) + i_batch + 1)
                score = roc_auc_score(y_true.detach().cpu().numpy(), y_pred.detach().sigmoid().cpu().numpy())
                print(f"batch <{i_batch}> - loss: {last_loss} - roc_auc score: {score}")
                running_loss = 0.
                
                #test_loss, test_roc_auc = test()
                #print(f"\tTest loss: {test_loss} \t Test ROC AUC: {test_roc_auc}")
                
                print(f"Metrics: {evaluate_nn(model, mp_matrix, val_matrix, k=10)}")
            
        print(f"Epoch: {epoch}, Loss: {running_loss / len(train_loader):.4f}")

In [22]:
def precision_k(reco_relevance, k=10):
    # k = reco_relevance.shape[1]
    # return (reco_relevance.sum(axis=1) / k).mean()
    return reco_relevance.mean()

def mean_average_prec(reco_relevance):
    K = reco_relevance.shape[1]
    
    mean_ap = 0.0
    for k in range(1, K+1):
        mean_ap += prec_k(reco_relevance[:, :k]) # DODAC MNOŻNIK 1/0 GDY ITEM JEST RELEWANTNY!!!
    return mean_ap / K

def recall_k(reco_relevance, relevance, k=10):
    sum_relevant = relevance.sum(axis=1)
    return (reco_relevance.sum(axis=1) / sum_relevant).mean()

@torch.no_grad()
def generate_embeddings(model, data):
    model.eval()
    data = data.to(device)
    x_dict = model.evaluate(data)
    return x_dict

@torch.no_grad()
def recommend_k(user_emb, item_emb, past_interactions=None, k=10, user_batch_size=1000):
    def remove_past_interactions(prob, user_batch):
        id_x = np.repeat(np.arange(user_batch.shape[0]), np.diff(past_interactions[user_batch].indptr))
        id_y = past_interactions[user_batch].indices
        prob[id_x, id_y] = -torch.inf
        return prob
    
    recommended_batches = []
    user_batches = torch.arange(user_emb.shape[0]).split(user_batch_size)
    for user_batch in user_batches:
        prob = (user_emb[user_batch] @ item_emb.T).sigmoid()
        prob = remove_past_interactions(prob, user_batch)
        recommended_batches.append(prob.topk(k, 1)[1])
    
    recommendations = torch.cat(recommended_batches, 0)
    return recommendations

def recommendation_relevance(recommendations, ground_truth):
    """
    Computes the relevance matrix of recommended items based on ground truth data.

    This function takes a matrix of recommended items and a ground truth sparse matrix, and calculates
    binary relevance of recommended items for each user. The relevance is determined by
    comparing the recommended items with the actual items in the ground truth.

    Args:
        recommendations (numpy.ndarray): A 2D matrix of shape (n_users, k) where k is the number of 
            recommended items per user. Each row contains indices representing the recommended 
            items for a user.
        ground_truth (scipy.csr_matrix): A sparse matrix of shape (n_users, n_items). The matrix 
            contains binary values indicating whether an item is relevant (1) or not (0) for each user.

    Returns:
        numpy.matrix: A 2D matrix of shape (n_users, k) containing the relevance scores of the
        recommended items for each user.
        
    Raises:
        ValueError: If the dimensions of 'recommendations' and 'ground_truth' do not match or
            are incompatible for matrix operations.
    """
    n_users, n_items = ground_truth.shape
    k = recommendations.shape[1]
    
    if recommendations.shape[0] != n_users:
        raise ValueError("Number of users in 'recommendations' should match 'ground_truth'.")
    
    user_idx = np.repeat(np.arange(n_users), k)
    item_idx = recommendations.flatten()
    relevance = ground_truth[user_idx, item_idx].reshape((n_users, k))  # get values under arrays of indices 
                                                                        # (user_idx and item_idx) from ground truth
    relevance_mask = np.asarray((ground_truth.sum(axis=1) != 0)).ravel()
    
    return relevance, relevance_mask

def evaluate_nn(model, mp_matrix, val_matrix, k):
    x_emb = generate_embeddings(model, val_data)
    recommendations = recommend_k(x_emb['user'], x_emb['app'], past_interactions=mp_matrix, 
                                  k=10, user_batch_size=10000).cpu().numpy()
    reco_relevance, relevance_mask = recommendation_relevance(recommendations, val_matrix)
    
    prec_k = precision_k(reco_relevance[relevance_mask], k)
    return {f"precision@{k}": prec_k}

In [23]:
@torch.no_grad()
def test():
    model.eval()
    running_loss = 0.
    preds, ground_truths = [], []

    for i_batch, batch in enumerate(val_loader):
        batch = batch.to(device)
        y_pred = model(batch)
        y_true = batch['user', 'recommends', 'app'].edge_label
        loss = criterion(y_pred, y_true.float())
        
        preds.append(y_pred)
        ground_truths.append(y_true)

        running_loss += loss.item()
        
    pred = torch.cat(preds, dim=0).cpu().numpy()
    ground_truth = torch.cat(ground_truths, dim=0).cpu().numpy()
    
    test_loss = running_loss / len(test_loader)
    test_score = roc_auc_score(ground_truth, pred)

    return test_loss, test_score

In [24]:
train(n_epochs=5, print_loss=100)

  5%|████▍                                                                                  | 99/1924 [01:05<18:29,  1.65it/s]

batch <99> - loss: 0.4053977537155151 - roc_auc score: 0.907811164855957


  5%|████▎                                                                               | 100/1924 [01:44<6:07:54, 12.10s/it]

Metrics: {'precision@10': 0.016886577}


 10%|████████▉                                                                             | 199/1924 [02:52<19:40,  1.46it/s]

...

Metrics: {'precision@10': 0.017870741}


 57%|████████████████████████████████████████████████▌                                    | 1099/1924 [19:29<11:47,  1.17it/s]

batch <1099> - loss: 0.24217766240239144 - roc_auc score: 0.9645195007324219


 57%|████████████████████████████████████████████████▌                                    | 1099/1924 [19:49<14:52,  1.08s/it]


KeyboardInterrupt: 

In [ ]:
p2 = retrieval.RetrievalPrecision(top_k=5)
r2 = retrieval.RetrievalRecall(top_k=2)
ndcg = retrieval.RetrievalNormalizedDCG(top_k=7)

In [ ]:
preds = torch.tensor([0.7, 0.8, 0.1, 0.2, 0.4, 0.6, 0.5, 0.9, 0.3, 0.15])
targets = torch.tensor([True, True, False, False, False, False, False, False, False, False])
indices = torch.tensor([0,0,0,0,0,0,0,0,0,0])

mask = torch.ones(preds.shape, dtype=torch.bool)
mask[[4, 5, 7]] = False
print(preds[mask])
print(targets[mask])

In [ ]:
preds = torch.tensor([1.0, 0.85, 0.8, 0.7, 0.65])
targets = torch.tensor([False, False, False, False, False])
indices = torch.tensor([0,0,0,0,0])

In [ ]:
p2(preds, targets, indexes=indices)

In [ ]:
r2(preds, targets, indexes=indices)

In [ ]:
ndcg(preds, targets, indexes=indices)

In [ ]:
def dcg(rel):
    g = 0.
    for i in range(1,6):
        g+= (2**rel[i-1] - 1)/np.log2(i+1)
    return g

In [ ]:
rel = [1,0,1,0,1]
rel_idcg = [1,1,1,0,0]
print(dcg(rel))
print(dcg(rel_idcg))
print(dcg(rel)/dcg(rel_idcg))

In [ ]:
torch.tensor([0.9, 0.7, 0.6])

In [ ]:
(preds[i][:3] / torch.log2(torch.arange(3)+2)).sum() / (torch.tensor([0.9, 0.7, 0.6]) / torch.log2(torch.arange(3)+2)).sum()

In [ ]:
torch.log2(torch.arange(3)+2)

In [ ]:
cm = confusion_matrix(y_true.detach().cpu().numpy(), y_pred.detach().cpu().numpy().round())
cm_display = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[False, True])
cm_display.plot()
plt.show()

In [25]:
def save_model(model, path):
    torch.save(model.state_dict(), path)
    
def load_model(path):
    model = Model(hidden_channels=32, out_channels=32, metadata=train_data.metadata())
    model.load_state_dict(torch.load(path))
    model = model.to(device)
    return model

In [26]:
save_model(model, "models/gnn_03.pth")

In [ ]:
model = load_model("models/gnn_02.pth")

In [ ]:
print(nn.summary(model, next(iter(train_loader)).to(device)))

In [ ]:
model